# Datasets

In this notebook, you will be guided to :
- Search your **project** and describe it
- Search your **mission** and describe it
- Search your **datasets** and describe them
- Display the particularity of vector dataset
- Search an **annotation**
- Launch an **analytic**

#### You may execute this notebook one cell after the other with `Shift + Enter`

For more options, explore `Cell` in the top menu bar, or read a [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/)

In [ ]:
import os
import alteia
import time

In [ ]:
import getpass
platform_url = 'https://app.alteia.com'
login = input('Enter your email ')
password = getpass.getpass('Enter your password ')

In [ ]:
sdk = alteia.SDK(url=platform_url, user=login, password=password)

In [ ]:
# Set working directory
sample_path = './Banana'
%cd {sample_path}

# Exercices

### Get your company id

In [ ]:
company_id = sdk.users.describe().companies[0]['id']
print(f"My company id is: {company_id}")

### Search your project

In [66]:
input(project_name) ### Add your project name
lst_proj = sdk.projects.search(
    filter={
        'company': {'$eq': company_id},
        'name': {'$eq': project_name}
    }
)

### Describe it and search creation date

In [ ]:
my_project = lst_proj[0]
my_project.__dict__
my_project.created

### Search mission in your project

In [ ]:
missions = sdk.missions.search(
  filter={
    'project': {'$eq': my_project.id},
  }
)

for m in missions:
  print(m.name, m.id, m.creation_date)

missions[0].__dict__
my_mission = missions[0]

### Search datasets in your project / mission

In [ ]:
lst_datasets = sdk.datasets.search(
  filter={
    'project': {'$eq': my_project.id},
    'mission': {'$eq': my_mission.id},
  }
)

print(len(lst_datasets))

### List datasets with their name, type, source, categories and ingestio status

In [ ]:
for d in lst_datasets:
  print(d.name, d.id, d.type, d.source, d.categories, d.ingestion['status'])

### Describe the vector dataset

In [ ]:
vct_id = '65b9216b2a7f24ad6a55c53c'
vct = sdk.datasets.describe(vct_id)

### Search an annotation

In [ ]:

# At project level
lst_annotations = sdk.annotations.search(
    project=my_project.id
)
print(f'Project annotation number: {len(lst_annotations)}')
# At mission level
lst_annotations = sdk.annotations.search(
    filter={
        'project': {'$eq': my_project.id},
        'mission': {'$eq': my_mission.id}
    }
)
print(f'Project and mission annotation number: {len(lst_annotations)}')

## Launch an analytic

In [ ]:
# Upload the vector Maison
vector_dataset = sdk.datasets.create_vector_dataset(
    name='Maison',
    project=my_project.id,
    mission=my_mission.id,
    dataset_format='geojson')

sdk.datasets.upload_file(
  dataset=vector_dataset.id,
  component='vector',
  file_path='../Maison.geojson')


In [ ]:
# Waiting for ingestion
while vector_dataset.ingestion['status'] != 'completed':
    vector_dataset = sdk.datasets.describe(vector_dataset.id)
    time.sleep(10)
    print("Waiting for ingestion... 10s")
print(f'Ingestion completed, vector id is: {vector_dataset.id}')

In [ ]:
# Search for orthomosaic
orthomosaic = sdk.datasets.search(
  filter={
    'project': {'$eq': my_project.id},
    'mission': {'$eq': my_mission.id},
    'name': {'$eq': 'Orthomosaic'},
    'categories': {'$in': ['orthomosaic']},
  }
)

if len(orthomosaic) == 1:
    orthomosaic = orthomosaic[0]
    print(f'Orthomais id is: {orthomosaic.id}')
else:
    print('Too many orthomosaic..., quit')

In [ ]:
# Launch the analytic
analytic = sdk.analytics.order(
    analytic='651294c21e362ddf74a0e605', # raster crop
    inputs={
        'raster': [orthomosaic.id],
        'aoi': [vector_dataset.id]
    },
    parameters={
        'suffix': 'Maison_buffer',
        'buffer': -10.0,
    },
    project=my_project.id,
    mission=my_mission.id
)
print('Analytic launched...')

In [ ]:
raster_crop = sdk.products.describe(analytic.id)
print(f'Raster crop deliverable is {raster_crop.deliverables}')